# Transformer training (chantier en cours)

Imports

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [2]:
from statapp.common.preprocessing import load_data, encode_data, split_into_X_y

In [3]:
from statapp.common.sampling import sample_token_sequence

Preprocessing maison bien moche pour le moment... ^^ (essai avec un encodage sur les mots) Les données sont placées directement dans le dossier du notebook

In [4]:
text = load_data("data/fr.train.top1M.txt", sample=0.0005)

tokens = nltk.word_tokenize(text)

In [5]:
len(tokens)

25115

In [6]:
vocab = list(set(tokens))

vocab_size = len(vocab)

vocab_numbers = dict(zip(vocab, range(0,len(vocab))))
vocab_numeroted = dict(zip(range(0,len(vocab)), vocab))
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

tokens_numbers_sequences = np.array([ tokens_numbers[i:i+max_length] for i in range(len(tokens_numbers)-max_length+1)])
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

nb_sequences =  tokens_numbers_sequences.shape[0]

In [7]:
vocab_size

6275

Apprentissage du modèle

In [8]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

#Correspond à utiliser l'entropie croisée puisque les sorties sont des log_softmax
#et l'entropie croisée = nll_loss(log_softmax(.), target)
criterion = nn.NLLLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [9]:
def train_model(nb_epochs, batch_size):
    
    #What is this ?? I don't remember. Make grad required ?
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for i, batch_indices in enumerate(batchs_indices):
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()
            
            #Il faudrait adapter les affichages en fonction du nombre de batchs total
            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.

Test d'overfitting sur un cas ultrasimplifié (5 tokens, longueur de séquence 1, 3 decoders, 2 heads) :
- En observant les sorties le modèle a bien appris et overfitte ! (loss à 0 au bout de 5-6 epochs) 

In [10]:
train_model(10,20)

C:\Users\Eric\statapp_language_model\statapp\transformer\pytorch\transformer_model.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.add(embedded, pos_encodings), dtype=torch.float32)


[1,   100] loss: 6.867
[1,   200] loss: 6.304
[1,   300] loss: 6.029
[1,   400] loss: 5.864
[1,   500] loss: 5.662
[1,   600] loss: 5.423
[1,   700] loss: 5.358
[1,   800] loss: 5.129
[1,   900] loss: 5.033
[1,  1000] loss: 4.898
[1,  1100] loss: 4.763
[1,  1200] loss: 4.674
[2,   100] loss: 4.163
[2,   200] loss: 4.121
[2,   300] loss: 4.100
[2,   400] loss: 4.057
[2,   500] loss: 4.043
[2,   600] loss: 3.992
[2,   700] loss: 3.953
[2,   800] loss: 3.897
[2,   900] loss: 3.828
[2,  1000] loss: 3.790
[2,  1100] loss: 3.772
[2,  1200] loss: 3.694
[3,   100] loss: 3.277
[3,   200] loss: 3.346
[3,   300] loss: 3.318
[3,   400] loss: 3.317
[3,   500] loss: 3.310
[3,   600] loss: 3.310
[3,   700] loss: 3.272
[3,   800] loss: 3.240
[3,   900] loss: 3.228
[3,  1000] loss: 3.174
[3,  1100] loss: 3.182
[3,  1200] loss: 3.168
[4,   100] loss: 2.726
[4,   200] loss: 2.812
[4,   300] loss: 2.770
[4,   400] loss: 2.819
[4,   500] loss: 2.857
[4,   600] loss: 2.823
[4,   700] loss: 2.804
[4,   800] 

Génération

In [11]:
#bidouilles pour adapter nos fonctions aux fonctions common codées par Nathra 
#(sequence list of ints en entree, list of probas en sortie)
#(Faire mieux plus tard)
def LMtransformerprediction(listints):
    return np.exp(LMtransformer(torch.tensor([listints]))[0][-1].tolist())

In [12]:
def gen_seq(prev_seq):
    with torch.no_grad():
        prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
        sample_token_seq = sample_token_sequence(LMtransformerprediction, prev_seq_numbers)
        tokens_pred = [vocab_numeroted[i] for i in sample_token_seq]
        print(' '.join(tokens_pred)) 

In [13]:
gen_seq(['il'])

100%|████████████████████████████████████████| 100/100 [00:07<00:00, 13.62it/s]


remporte le club a plusieurs annees 1970 pour un personnage est declare et la guerre et un personnage 2012 national . ainsi a percevoir , il y maintenir trois grandes entre le `` comme `` a une plage `` la , dans l ' en afrique de l ' `` et que les , le realisateur `` comme `` ) , dans le 10 . au `` , les annees nous sa `` ( `` , il `` , le premier `` , ne connurent un personnage `` , le 13 , le 13 , la , les etats-unis ``
